# Compress and Evaluate Large Language Models

<a target="_blank" href="https://colab.research.google.com/github/PrunaAI/pruna/blob/v|version|/docs/tutorials/llms.ipynb">
    <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

| Component | Details |
|-----------|---------|
| **Goal** | Show a standard workflow for optimizing and evaluating a large language model |
| **Model** | [HuggingFaceTB/SmolLM2-360M-Instruct](https://huggingface.co/HuggingFaceTB/SmolLM2-360M-Instruct) |
| **Dataset** | [SmolSmolTalk](https://huggingface.co/datasets/HuggingFaceTB/smol-smoltalk) |
| **Libraries** | [transformers](https://github.com/huggingface/transformers), [datasets](https://github.com/huggingface/datasets) |
| **Device** | 1 x RTX A5000 |
| **Optimization Algorithms** | quantizer(hqq), compiler(torch_compile) |
| **Evaluation Metrics** | perplexity, latency, energy consumption |

## Getting Started

### Install the dependencies

To install the dependencies, run the following command:

In [ ]:
!pip install pruna

### Set the device

Normally, we would set the device to the best available device to make the most out of the optimization process.


In [3]:
import torch

device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"

## Load the model

Before we can optimize the model, we need to ensure that we can load the model and tokenizer correctly and that they can fit in memory. For this example, we will use a nice and small LLM, [SmolLM2-360M-Instruct](https://huggingface.co/HuggingFaceTB/SmolLM2-360M-Instruct), but feel free to use any [text-generation model on Hugging Face](https://huggingface.co/models?pipeline_tag=text-generation). 

Although Pruna works at least as good with smaller models, a small model is a good starting point to show the steps of the optimization process.

In [ ]:
from transformers import pipeline

model_name = "HuggingFaceTB/SmolLM2-360M-Instruct"

pipe = pipeline(
    task="text-generation",
    model=model_name,
)

Device set to use cuda:0


Now we've loaded the model and tokenizer. Let's see if we can run some inference with them. To make this easy for use, we will be using the `transformers` library's `pipeline.__call__` function and passing in a list of messages.

In [5]:
from transformers import pipeline

messages = [
    {"role": "user", "content": "Who are you?"},
]
pipe(messages, max_new_tokens=100)

[{'generated_text': [{'role': 'user', 'content': 'Who are you?'},
   {'role': 'assistant',
    'content': "I'm a chatbot developed by Hugging Face, designed to provide users with helpful and accurate information about various topics. I was trained using a massive corpus of text data, including texts from reputable sources such as Wikipedia, news articles, and academic journals. My primary goal is to assist users in understanding complex concepts and providing concise answers to their questions."}]}]

As we can see, the model is able to generate a response to the user's question, which is being cut-off after the allowed `max_new_tokens`. 

## Define the SmashConfig

Now we know the model is working, let's continue with the optimization process and define the `SmashConfig`, which we will use later on to optimize the model.

Not all optimization algorithms are available for all models but we can learn a bit more about different optimization algorithms and their requirements in the [Algorithms Overview](https://docs.pruna.ai/en/stable/compression.html) section of the documentation.

For the current optimization, we will be using the [`hqq` quantizer](https://docs.pruna.ai/en/stable/compression.html#hqq) and the [`torch_compile` compiler](https://docs.pruna.ai/en/stable/compression.html#torch-compile). We will updating some parameters for these algorithms, setting `hqq_weight_bits` to `4`, `hqq_compute_dtype` to `torch.bfloat16`, `torch_compile_fullgraph` to `True`, `torch_compile_dynamic` to `True`, and `torch_compile_mode` to `max-autotune`. This is one of the many configurations and will just serve as an example.

Let's define the `SmashConfig` object.

In [42]:
from pruna import SmashConfig

smash_config = SmashConfig(device=device)
smash_config["quantizer"] = "hqq"
smash_config["hqq_weight_bits"] = 8
smash_config["hqq_compute_dtype"] = "torch.bfloat16"
# We can also use `torch_compile` as our compiler, but we will skip it for now as it will take a bit longer to compile.
# smash_config["compiler"] = "torch_compile"
# smash_config["torch_compile_fullgraph"] = True
# smash_config["torch_compile_dynamic"] = True
# smash_config["torch_compile_mode"] = "max-autotune"

## Smash the model

Now that we have defined the `SmashConfig` object, we can smash the model. We will be using the `smash` function to smash the model and pass the `model` and `smash_config` to it. We also make a deep copy of the model to avoid modifying the original model. 

Let's smash the model, which should take around 20 seconds for this configuration.

In [44]:
import copy

from pruna import smash

copy_model = copy.deepcopy(pipe.model)
smashed_model = smash(
    model=pipe.model,
    smash_config=smash_config,
)

INFO - Starting quantizer hqq...
100%|██████████| 225/225 [00:06<00:00, 36.66it/s]
INFO - quantizer hqq was applied successfully.
INFO - Starting compiler torch_compile...
ERROR - max-autotune-no-cudagraphs is not supported for causal language models.
INFO - compiler torch_compile was applied successfully.


Now we've optimized the model. Let's see if everything still works as expected and we can run some inference with the optimized model. In this case, we are running the inference by first encoding the prompt through the `tokenizer` and then passing the `input_ids` to the `PrunaModel.generate` method, which also allows us to specify additional parameters such as `max_new_tokens`.

If you are using `torch_compile` as your compiler, you can expect the first inference warmup to take a bit longer than the actual inference.

In [53]:
prompt = "Who are you?"
messages = [{"role": "user", "content": prompt}]
pipe(messages, max_new_tokens=100)

WARNING - Unhandled kwargs in generate method: {'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0'), 'generation_config': GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "max_new_tokens": 256,
  "pad_token_id": 2,
  "temperature": 0.7
}
}


[{'generated_text': [{'role': 'user', 'content': 'Who are you?'},
   {'role': 'assistant',
    'content': "I'm a helpful AI assistant from Hugging Face. What can I help you with today?"}]}]

As we can see, the model is able to generate a similar response to the original model. 

If you notice a significant difference, it might have several reasons, the models, the configuration, the hardware, etc. As optimization can be non-deterministic, we encourage you to retry the optimization process or try out different configurations and models to find the best fit for your use case but also feel free to reach out to us on [Discord](https://discord.gg/Tun8YgzxZ9) if you have any questions or feedback.

## Evaluate the smashed model

Now that we have optimized the model, we can evaluate the performance of the optimized model. We will be using the `EvaluationAgent` to evaluate the performance of the optimized model. We will do so with some basic metrics, the `elapsed_time`, as well as a stateful metrics, the `perplexity`. An overview of the different metrics can be found in our [documentation](https://docs.pruna.ai/).

Let's define the `EvaluationAgent` object and start the evaluation process. Note that we are using the `datamodule.limit_datasets(10)` method to limit the number of datasets to 10, which is just for the sake of time. Additionally, we lower the `n_iterations` and `n_warmup_iterations` to ensure that we monitor the performance of the model whenever it is running smoothly.

The evaluation can take anywhere from a couple of minutes to a couple of hours to complete, depending on your hardware, the number of samples in the dataset, and the configuration of the model. In our case it should only take a couple of minutes.

In [ ]:
from pruna import PrunaModel
from pruna.data.pruna_datamodule import PrunaDataModule
from pruna.evaluation.evaluation_agent import EvaluationAgent
from pruna.evaluation.metrics import EnergyConsumedMetric, LatencyMetric, TorchMetricWrapper
from pruna.evaluation.task import Task

# Define the metrics
metrics = [
    TorchMetricWrapper("perplexity", call_type="pairwise"),
    LatencyMetric(n_iterations=10, n_warmup_iterations=5),
    EnergyConsumedMetric(n_iterations=10, n_warmup_iterations=5),
]

# Define the datamodule
pipe.tokenizer.pad_token = pipe.tokenizer.eos_token
datamodule = PrunaDataModule.from_string("SmolSmolTalk", tokenizer=pipe.tokenizer)
datamodule.limit_datasets(100)

# Define the task and evaluation agent
task = Task(metrics, datamodule=datamodule, device=device)
eval_agent = EvaluationAgent(task)

# Evaluate base model, and smashed model
base_model = PrunaModel(model=copy_model)
smashed_model = pipe.model
inference_args = {"max_new_tokens": 500}
base_model.inference_handler.model_args.update(inference_args)
smashed_model.inference_handler.model_args.update(inference_args)
base_model_results = eval_agent.evaluate(base_model)
smashed_model_results = eval_agent.evaluate(smashed_model)

Now we can see the results of the evaluation and compare the performance of the original and the optimized model.

In [36]:
# Calculate percentage differences for each metric
def calculate_percentage_diff(original, optimized):  # noqa: D103
    return ((optimized - original) / original) * 100


# Calculate and display percentage differences
print("Percentage differences between original and optimized model:")
for base_metric_result, smashed_metric_result in zip(base_model_results, smashed_model_results):
    diff = calculate_percentage_diff(base_metric_result.result, smashed_metric_result.result)
    print(f"{base_metric_result.name}: {base_metric_result.result} vs {smashed_metric_result.result}: {diff:.2f}%")

Percentage differences between original and optimized model:
perplexity: 41.82640838623047 vs 46.976871490478516: 12.31%
latency: 1034.3694274902343 vs 200.73682022094727: -80.59%
energy_consumed: 0.0019644932473006573 vs 0.001022020140780868: -47.98%


As we can see, the optimized model is roughly 4x faster and consume only 1/2 of the energy of the base model, while losing some of its performance on perplexity, which is expected given the nature of the optimization. Now, we can start to compare, iterate and see what optimization works best for our models, given the metrics we are interested in.

We can now save the optimized model to disk and share it with others. Note that some optimizations, such as `torch_compile`, are device dependent and will be re-applied when loading the model on a different device.

In [ ]:
smashed_model.save_pretrained("smashed_model")
smashed_model.save_to_hub("smashed_model")

## Conclusion

In this tutorial, we have shown a standard workflow for optimizing and evaluating a large language model. We have used the `SmashConfig` object to define the optimization algorithms and the `EvaluationAgent` to evaluate the performance of the optimized model. We have also used the `PrunaDataModule` to load the dataset and the `Task` object to define the task and evaluation agent.

We have shown how to optimize the model using the `smash` function and how to evaluate the performance of the optimized model using the `EvaluationAgent`.

Proving we can optimize the model, by making it quicker, more energy efficient and using less memory, while only losing a small amount of accuracy.